In [ ]:
pip install google-cloud-translate==2.0.1

In [30]:
import os
from google.cloud import translate_v2 as translate
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key/key.json'

def translate_text_with_model(target:str, text:str, model="nmt")->str:
    """Translates text into the target language.

    Make sure your project is allowlisted.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target, model=model)

    # print(u"Text: {}".format(result["input"]))
    # print(u"Translation: {}".format(result["translatedText"]))
    # print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

    return result["translatedText"]

In [26]:
import pandas as pd

train = pd.read_csv('dataset/train/train.csv')

In [28]:
def back_translation(sentence:str, langs:list)->str:
    sent_tr = translate_text_with_model(langs[0], sentence)
    for i in range(1, len(langs)):
        sent_tr = translate_text_with_model(langs[i], sent_tr)           
    sent_btr = translate_text_with_model("ko", sent_tr)
    return sent_btr

In [46]:
langs_j_e = ["ja", "en"]
langs_j_e_f = ["ja", "en", "fr"]
data_bt = {'id':[], 'sentence':[], 'subject_entity':[], 'object_entity':[], 'label':[], 'source':[]}

for i in [16082, 30987, 2472, 2487]:
    sent = train.iloc[i]['sentence']
    subj = eval(train.iloc[i]['subject_entity'])
    obj = eval(train.iloc[i]['object_entity'])
    label = train.iloc[i]['label']
    
    subj_s = subj['start_idx']
    subj_e = subj['end_idx']
    obj_s = obj['start_idx']
    obj_e = obj['end_idx']
    subj_type = subj['type']
    obj_type = obj['type']
    
    print(sent)
    print(subj, obj)
    print(label)

    if subj_s < obj_s:
      sent_blocked = sent[:subj_s]+'[SUB]'+sent[subj_e+1:obj_s]+'[OBJ]'+sent[obj_e+1:]
    elif obj_s < subj_s:
      sent_blocked = sent[:obj_s]+'[OBJ]'+sent[obj_e+1:subj_s]+'[SUB]'+sent[subj_e+1:]
    else:
      sent_blocked = sent
    # print('\n')
    print(sent_typed)
    
    print(back_translation(sent_blocked, langs_j_e))
    print(back_translation(sent_blocked, langs_j_e_f))
    print("\n")
    data_bt['id'].append(train.iloc[i]['id'])
    data_bt['sentence'].append(back_translation(sent_blocked, lgs_j_e))
    data_bt['subject_entity'].append(subj)
    data_bt['object_entity'].append(obj)
    data_bt['label'].append(label)
    data_bt['source'].append(train.iloc[i]['source'])
    

그러나 서재필은 휴전을 못보고 한국 전쟁이 한창이던 1951년 1월 5일 펜실베이니아주 필라델피아 노리스타운 몽고메리 병원 병실에서 후두암과 방광암, 과로의 합병증으로 일생을 마쳤다.
{'word': '서재필', 'start_idx': 4, 'end_idx': 6, 'type': 'PER'} {'word': '펜실베이니아주 필라델피아', 'start_idx': 41, 'end_idx': 53, 'type': 'LOC'}
per:place_of_death
그 결과로서 밴스는 [SUB]와 [OBJ] 행정부에서 직위들의 계승에 근무하였다.
그러나 [SUB]는 한국전쟁이 한창이던 1951년 1월 5일 노리스 타운 몽고메리 병원 병실에서 후두암, 방광암, 과로 등의 합병증으로 생을 마감했다.
그러나 [SUB]는 한국 전쟁이 휴전 중이던 1951년 1월 5일 노리스 타운 몽고메리 병원 병실에서 후두암, 방광암, 과로 등의 합병증으로 생을 마감했다.


1593년 6월 2차 진주성 전투에서 진주성이 함락되자 의병장인 김천일·고종후와 함께 촉석루에 올라 임금의 처소가 있는 북쪽을 향해 4배(四拜)를 한 후 스스로 진주 남강에 투신해 순절했다.
{'word': '고종후', 'start_idx': 40, 'end_idx': 42, 'type': 'PER'} {'word': '진주성', 'start_idx': 12, 'end_idx': 14, 'type': 'POH'}
per:place_of_death
그 결과로서 밴스는 [SUB]와 [OBJ] 행정부에서 직위들의 계승에 근무하였다.
1593년 6월 2차[OBJ]전투에서 진주성이 함락되자, 대장군인 Kim Tenichi[SUB]와 함께 단세키로로 올라가 왕의 거처가 있는 북쪽을 향해 4배로 진격했다. 그 자신이 진주난강에 투자했고 순수했다.
1593년 6월 2차 전투[OBJ]에서 진주성이 함락되자 유덕한 장군 김정일[SUB]과 함께 단세키로에 올라 왕의 거처가 있던 북쪽으로 4배, 그 다음에는 자기 자신이었다. 진주난강에 대한 순

In [47]:
train_bt = pd.DataFrame(data_bt)
train_bt.to_csv("train_bt.csv", index=False)